In [11]:
mesh

PolyData (0x20c17121f60)
  N Cells:    19602
  N Points:   10000
  N Strips:   0
  X Bounds:   -1.000e+01, 1.000e+01
  Y Bounds:   -2.309e-14, 2.000e+01
  Z Bounds:   1.416e-01, 7.483e+00
  N Arrays:   2

In [ ]:
import pyvista as pv
from pyvista import examples

plotter = pv.Plotter()
mesh = examples.load_random_hills()

actor = plotter.add_mesh(mesh, cmap='terrain', show_scalar_bar=False)
plotter.show()

ValueError: Invalid colormap "Elevation"

In [1]:
from src.data.vtu_h5_rev import vtu_to_h5

vtu_to_h5(vtu_file_directory='./dataset/full', 
          output_h5_dir='./dataset/new_h5_files', 
          vtu_array_name='Velocity', 
          train_ratio=0.9, 
          scaling_type=4, 
          scaler_name='standard', 
          overwrite=True)

Pass 1: Reading all VTU files and extracting data...


Reading VTUs: 100%|██████████| 358/358 [00:01<00:00, 238.00it/s]


Found variables: ['Ux', 'Uy', 'Pressure', 'Cp', 'Cf']
Fitting scalers for each variable...
Fitting scaler for Ux...
  Training data shape for Ux: torch.Size([4693472, 1])
  Scaler fitted for Ux
Fitting scaler for Uy...
  Training data shape for Uy: torch.Size([4693472, 1])
  Scaler fitted for Uy
Fitting scaler for Pressure...
  Training data shape for Pressure: torch.Size([4693472, 1])
  Scaler fitted for Pressure
Fitting scaler for Cp...
  Training data shape for Cp: torch.Size([4693472, 1])
  Scaler fitted for Cp
Fitting scaler for Cf...
  Training data shape for Cf: torch.Size([4693472, 2])
  Scaler fitted for Cf
Skipping scaling for now - using unscaled data...
Scalers saved to ./dataset/new_h5_files\scaler.pkl (not currently used)
Writing training data to ./dataset/new_h5_files\train.h5 (322 trajectories)


Writing train.h5: 100%|██████████| 322/322 [00:00<00:00, 702.07it/s]


Writing validation data to ./dataset/new_h5_files\val.h5 (36 trajectories)


Writing val.h5: 100%|██████████| 36/36 [00:00<00:00, 793.88it/s]

VTU to H5 conversion with scaling and splitting complete. Output in ./dataset/new_h5_files


('./dataset/new_h5_files\\train.h5',
 './dataset/new_h5_files\\val.h5',
 './dataset/new_h5_files\\scaler.pkl')

In [ ]:
import pickle

with open('./dataset/new_h5_files/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

[StandardScaler(), StandardScaler()]

# Apply scaling

In [ ]:
from src.data.apply_scaling import load_scalers, apply_scaling_to_h5_dataset

# Load the scalers that were saved during VTU conversion
scalers = load_scalers('./dataset/new_h5_files/scaler.pkl')

# Apply scaling to Ux only
apply_scaling_to_h5_dataset(
    h5_file_path='./dataset/new_h5_files/train.h5',
    variable_name='Ux',
    scaler=scalers['Ux'],
    scaling_type=4,  # SAMPLE-FEATURE SCALING
    output_path='./dataset/new_h5_files/train_scaled.h5'
)

In [1]:
from src.data.apply_scaling import load_scalers, apply_scaling_to_all_variables

# Load scalers
scalers = load_scalers('./dataset/new_h5_files/scaler.pkl')

# Apply scaling to all variables (now using the working approach)
apply_scaling_to_all_variables(
    h5_file_path='./dataset/new_h5_files/train.h5',
    scalers=scalers,
    scaling_type=4,
    output_path='./dataset/new_h5_files/train_scaled.h5'
)

Applying scaling to each variable...
Scaling Ux...
Scaling Uy...
Scaling Pressure...
Scaling Cp...
Scaling Cf...
Writing scaled data to H5 file...


'./dataset/new_h5_files/train_scaled.h5'

In [2]:
from src.data.apply_scaling import get_variable_statistics

# Check statistics before scaling
stats_before = get_variable_statistics('./dataset/new_h5_files/train.h5', 'Ux')
print(f"Before scaling - Min: {stats_before['min']:.6f}, Max: {stats_before['max']:.6f}")

# After scaling
stats_after = get_variable_statistics('./dataset/new_h5_files/train_scaled.h5', 'Ux')
print(f"After scaling - Min: {stats_after['min']:.6f}, Max: {stats_after['max']:.6f}")

Before scaling - Min: -0.867209, Max: 2.129537
After scaling - Min: -6.522560, Max: 6.528021


In [ ]:
# 1. Load your scalers
scalers = load_scalers('./dataset/new_h5_files/scaler.pkl')

# 2. Your model gives scaled predictions
model_scaled_output = your_model(scaled_input)  # Shape: [num_nodes, 1]

# 3. Inverse scale to get real-world values
real_world_prediction = inverse_scale_single_trajectory(
    model_scaled_output, 
    scalers['Ux'], 
    scaling_type=4
)

# Now real_world_prediction contains values in the original scale!